In [29]:
import pandas as pd
import numpy as np

# Computing Base Rates and Compiling Final Dataset

In [30]:
# Load data
all_events = pd.read_excel('2024 Olympics Sports and Events.xlsx')

# Convert to correct data types
binary_columns = ['US Gold 20', 'US Silver 20', 'US Bronze 20',
                 'US Gold 16', 'US Silver 16', 'US Bronze 16',
                  'New']
all_events[binary_columns] = all_events[binary_columns].apply(pd.to_numeric, errors='coerce')
all_events.head()

# Fill Na values with 0
all_events = all_events.fillna(0)

# Display dataframe
all_events.head()

,Sport,Event,US Gold 20,US Silver 20,US Bronze 20,US Gold 16,US Silver 16,US Bronze 16,New
0,Archery,Men's Individual,0.0,0.0,0.0,0.0,0.0,1.0,0
1,Archery,Women's Individual,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Archery,Men's Team,0.0,0.0,0.0,0.0,1.0,0.0,0
3,Archery,Women's Team,0.0,0.0,0.0,0.0,0.0,0.0,0
4,Archery,Mixed Team,0.0,0.0,0.0,0.0,0.0,0.0,0


In [31]:
# Create point columns for each year for each event
all_events['2020 Points'] = (all_events['US Gold 20'] + (1/10)*all_events['US Silver 20'] + \
                              (1/20)*all_events['US Bronze 20']) / 1.15
all_events['2016 Points'] = (all_events['US Gold 16'] + (1/10)*all_events['US Silver 16'] + \
                             (1/20)*all_events['US Bronze 16']) / 1.15
all_events[all_events.isna().any(axis=1)]

,Sport,Event,US Gold 20,US Silver 20,US Bronze 20,US Gold 16,US Silver 16,US Bronze 16,New,2020 Points,2016 Points


In [32]:
# Get the number of events for each sport category
events_count = all_events.groupby("Sport").count()[['Event']].reset_index()
events_count = events_count.rename(columns = {"Event":"Number of Events"})
events_count.head()

,Sport,Number of Events
0,Archery,5
1,Artistic Gymnastics,14
2,Artistic Swimming,2
3,Athletics,48
4,Badminton,5


In [33]:
# Get the mean points in each sport for 2016 and 2020, and also compute weighted average
medal_prop_by_sport = all_events.drop(columns = ['Event'])\
    .groupby('Sport')\
    .mean()[['2020 Points', '2016 Points']]
medal_prop_by_sport['Weighted Average'] = 0.8*medal_prop_by_sport['2020 Points'] + \
      0.2*medal_prop_by_sport['2016 Points']
medal_prop_by_sport.sort_values(by = "Weighted Average", ascending = False).head()

,2020 Points,2016 Points,Weighted Average
Sport,,,
Basketball,0.869565,0.869565,0.869565
Golf,0.869565,0.021739,0.700000
Waterpolo,0.434783,0.434783,0.434783
Volleyball,0.434783,0.043478,0.356522
Beach Volleyball,0.434783,0.021739,0.352174


In [34]:
# Lump together all sports with 0 weighted average into an "Other" category
table_other = medal_prop_by_sport[medal_prop_by_sport['Weighted Average'] != 0].reset_index()
table_other.loc[len(table_other.index)] = ['Other', 0,0,0]
table_other = table_other.sort_values('Weighted Average', ascending=False) \
    .reset_index() \
    .drop(columns=['index'])
table_other.tail()

,Sport,2020 Points,2016 Points,Weighted Average
28,Weightlifting,0.013043,0.004348,0.011304
29,Tennis,0.000000,0.034783,0.006957
30,Archery,0.000000,0.026087,0.005217
31,Sailing,0.000000,0.004348,0.000870
32,Other,0.000000,0.000000,0.000000


In [35]:
# Get the total number of events in the sports lumped together into the "Other" category
other = medal_prop_by_sport[medal_prop_by_sport['Weighted Average'] == 0]
other_counts = other.merge(events_count, on = 'Sport', how = 'left')
other_impute = np.sum(other_counts['Number of Events'])

# Update the events count table
events_count = events_count._append({'Sport':'Other',
                                     'Number of Events':other_impute}, ignore_index=True)
events_count.tail()

,Sport,Number of Events
41,Volleyball,2
42,Waterpolo,2
43,Weightlifting,10
44,Wrestling,18
45,Other,36


In [36]:
# Construct final table by merging the number of events for each sport
final_table = table_other.merge(events_count, on = "Sport", how = "left")
final_table.head()

,Sport,2020 Points,2016 Points,Weighted Average,Number of Events
0,Basketball,0.869565,0.869565,0.869565,2
1,Golf,0.869565,0.021739,0.700000,2
2,Waterpolo,0.434783,0.434783,0.434783,2
3,Volleyball,0.434783,0.043478,0.356522,2
4,Beach Volleyball,0.434783,0.021739,0.352174,2


In [37]:
# Download the .csv
final_table.to_csv('final_forecast_dataset.csv')